In [24]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Deep Learning Libraries
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

In [25]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
def accuracy_loss_plot(history):
    plt.Figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='lower right')
    plt.show()
    plt.Figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='lower right')
    plt.show()

In [27]:
data = np.load('/content/drive/MyDrive/mnist/fashion_mnist_dataset_train.npy', allow_pickle=True).item()

In [28]:
s = pd.Series(data['target'])
target = s.replace([1, 2, 3, 4, 5], [0, 1, 2, 3, 4])
target = target.to_numpy()
target

array([4., 1., 0., ..., 2., 0., 3.])

In [29]:
from sklearn.model_selection import train_test_split
# Splitting the data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(data['features'], target, test_size=0.2, random_state=275)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=275)

In [30]:
print(X_val.shape, '\n')

(6000, 28, 28) 



In [31]:
#pad the images to achieve 32 x 32
X_train = tf.pad(X_train, [[0, 0], [2,2], [2,2]])
X_val = tf.pad(X_val, [[0, 0], [2,2], [2,2]])
X_test = tf.pad(X_test, [[0, 0], [2,2], [2,2]])
#expand and repeat to create 3 channels
X_train = tf.expand_dims(X_train, axis=3, name=None)
X_val = tf.expand_dims(X_val, axis=3, name=None)
X_test = tf.expand_dims(X_test, axis=3, name=None)


In [32]:
print(X_train.shape, '\n')
print(y_train.shape, '\n')
print(X_val.shape, '\n')
print(X_test.shape, '\n')

(48000, 32, 32, 1) 

(48000,) 

(6000, 32, 32, 1) 

(6000, 32, 32, 1) 



In [33]:
print(tf.__version__)

2.4.1


In [34]:
X_train = tf.repeat(X_train, 3, axis=3)
X_val = tf.repeat(X_val, 3, axis=3)
X_test = tf.repeat(X_test, 3, axis=3)

In [35]:
print(X_train.shape, '\n')
print(y_train.shape, '\n')
print(X_val.shape, '\n')
print(X_test.shape, '\n')

(48000, 32, 32, 3) 

(48000,) 

(6000, 32, 32, 3) 

(6000, 32, 32, 3) 



In [36]:
base_model = tf.keras.applications.VGG16(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model.layers:
  layer.trainable = False
base_model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [37]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(5, activation = 'softmax')(x)
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [38]:
head_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [39]:
with tf.device('/device:GPU:0'):
  history = head_model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100
375/375 [==============================] - 10s 26ms/step - loss: 0.8789 - accuracy: 0.6629 - val_loss: 0.5194 - val_accuracy: 0.7835
Epoch 2/100
375/375 [==============================] - 9s 25ms/step - loss: 0.5285 - accuracy: 0.7857 - val_loss: 0.4674 - val_accuracy: 0.8055
Epoch 3/100
375/375 [==============================] - 9s 25ms/step - loss: 0.4884 - accuracy: 0.8017 - val_loss: 0.4349 - val_accuracy: 0.8242
Epoch 4/100
375/375 [==============================] - 9s 25ms/step - loss: 0.4687 - accuracy: 0.8089 - val_loss: 0.3975 - val_accuracy: 0.8360
Epoch 5/100
375/375 [==============================] - 9s 25ms/step - loss: 0.4472 - accuracy: 0.8192 - val_loss: 0.3961 - val_accuracy: 0.8380
Epoch 6/100
375/375 [==============================] - 9s 25ms/step - loss: 0.4393 - accuracy: 0.8200 - val_loss: 0.3977 - val_accuracy: 0.8398
Epoch 7/100
375/375 [==============================] - 9s 25ms/step - loss: 0.4252 - accuracy: 0.8274 - val_loss: 0.3860 - val_accuracy

In [40]:
head_model.evaluate(X_test, y_test)

188/188 [==============================] - 2s 10ms/step - loss: 0.3776 - accuracy: 0.8558


[0.3775951862335205, 0.8558333516120911]